In [1]:
!pip install moviepy
!pip install librosa
!pip install noisereduce
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.3/388.3 kB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.9/26.9 MB 45.9 MB/s eta 0:00:00
  Created wheel for moviepy: filename=moviepy-1.0.3-py3-none-any.whl size=110721 sha256=7d7f8f39fe961a1c175994782c746e056556f10fce0e18d3799f4f4848367f19
  Stored in directory: /root/.cache/pip/wheels/96/32/2d/e10123bd88fbfc02fed53cc18c80a171d3c87479ed845fa7c1
Successfully built moviepy
  Attempting uninstall: decorator
    Found existing installation: decorator 5.1.1
    Uninstalling decorator-5.1.1:
      Successfully uninstalled decorator-5.1.1


In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import random
import os
from moviepy.editor import VideoFileClip
import librosa 
from transformers import DistilBertTokenizer, DistilBertModel
import torch
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.neural_network import MLPRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from scipy import stats 
import os
import cv2
import numpy as np
import pandas as pd
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, LSTM, TimeDistributed
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from transformers import AutoTokenizer, AutoModel
from transformers import RobertaTokenizer, RobertaModel
import noisereduce as nr
from scipy import signal


In [3]:
train_df = pd.read_csv('/kaggle/input/ml-hackathon-ec-campus-set-2/set_2_train/train_emotion.csv', 
                       encoding='ISO-8859-1')
video_dir = '/kaggle/input/ml-hackathon-ec-campus-set-2/set_2_train/train_data'


# Function to get video file path from IDs
def get_video_clip_path(row):
    dialogue_id = row['Dialogue_ID']
    utterance_id = row['Utterance_ID']
    filename = f"dia{dialogue_id}_utt{utterance_id}.mp4"
    return os.path.join(video_dir, filename)

# Apply the function to get file paths for each sampled clip
train_df['video_clip_path'] = train_df.apply(get_video_clip_path, axis=1)

# Check sample paths
print(train_df[['Dialogue_ID', 'Utterance_ID', 'video_clip_path']].head())

   Dialogue_ID  Utterance_ID  \
0            0             7   
1            0            11   
2            2             8   
3            3             3   
4            3             5   

                                     video_clip_path  
0  /kaggle/input/ml-hackathon-ec-campus-set-2/set...  
1  /kaggle/input/ml-hackathon-ec-campus-set-2/set...  
2  /kaggle/input/ml-hackathon-ec-campus-set-2/set...  
3  /kaggle/input/ml-hackathon-ec-campus-set-2/set...  
4  /kaggle/input/ml-hackathon-ec-campus-set-2/set...  


In [4]:
train_df.shape

(1000, 11)

In [5]:
emotions = train_df['Emotion']
emotions

0      neutral
1      neutral
2      neutral
3      neutral
4      neutral
        ...   
995    neutral
996        joy
997        joy
998    sadness
999    neutral
Name: Emotion, Length: 1000, dtype: object

In [6]:
# Define path to video clips
test_df = pd.read_csv('/kaggle/input/ml-hackathon-ec-campus-set-2/set_2_test/test_emotion.csv', encoding='ISO-8859-1')
test_video_dir = '/kaggle/input/ml-hackathon-ec-campus-set-2/set_2_test/test_data'


# Function to get video file path from IDs
def get_video_clip_path(row):
    dialogue_id = row['Dialogue_ID']
    utterance_id = row['Utterance_ID']
    filename = f"dia{dialogue_id}_utt{utterance_id}.mp4"
    return os.path.join(test_video_dir, filename)

# Apply the function to get file paths for each sampled clip
test_df['video_clip_path'] = test_df.apply(get_video_clip_path, axis=1)

# Check sample paths
print(test_df[['Dialogue_ID', 'Utterance_ID', 'video_clip_path']].head())

   Dialogue_ID  Utterance_ID  \
0            3             0   
1            5             7   
2           13             2   
3           13             6   
4           14             1   

                                     video_clip_path  
0  /kaggle/input/ml-hackathon-ec-campus-set-2/set...  
1  /kaggle/input/ml-hackathon-ec-campus-set-2/set...  
2  /kaggle/input/ml-hackathon-ec-campus-set-2/set...  
3  /kaggle/input/ml-hackathon-ec-campus-set-2/set...  
4  /kaggle/input/ml-hackathon-ec-campus-set-2/set...  


# for text:

In [7]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

inputs = tokenizer(list(train_df['Utterance']), return_tensors="pt", padding=True, truncation=True)
with torch.no_grad():
    outputs = model(**inputs)
    last_hidden_states = outputs.last_hidden_state
    
# Pool embeddings (mean) and convert to DataFrame
mean_embeddings = torch.mean(last_hidden_states, dim=1)
embedding_df = pd.DataFrame(mean_embeddings.numpy())
embedding_df.to_csv("output_embeddings.csv", index=False)
    
text_emb_df = pd.read_csv("output_embeddings.csv")
    
text_emb_df['label'] = train_df['Emotion']

X = text_emb_df.drop(columns=['label']).values  # Features
y = text_emb_df['label'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model1 = MLPClassifier(hidden_layer_sizes=(100, 50),  # Example: 2 hidden layers
                      activation='logistic', 
                      solver='adam', 
                      max_iter=75,  
                      random_state=42)


model1.fit(X_train, y_train)

y_pred = model1.predict(X_test)

# Calculate accuracy (for classification)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy) 

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Accuracy: 0.625


In [8]:
y_pred

array(['neutral', 'anger', 'neutral', 'surprise', 'neutral', 'neutral',
       'neutral', 'neutral', 'neutral', 'surprise', 'joy', 'neutral',
       'neutral', 'anger', 'neutral', 'surprise', 'surprise', 'neutral',
       'neutral', 'anger', 'neutral', 'neutral', 'neutral', 'joy',
       'surprise', 'anger', 'neutral', 'neutral', 'neutral', 'neutral',
       'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral',
       'anger', 'neutral', 'neutral', 'joy', 'neutral', 'neutral', 'joy',
       'surprise', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral',
       'neutral', 'joy', 'joy', 'surprise', 'surprise', 'neutral',
       'neutral', 'joy', 'surprise', 'neutral', 'surprise', 'neutral',
       'neutral', 'neutral', 'neutral', 'surprise', 'neutral', 'neutral',
       'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'surprise',
       'neutral', 'neutral', 'joy', 'neutral', 'neutral', 'joy',
       'surprise', 'joy', 'neutral', 'surprise', 'neutral', 'surprise',
  

In [9]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')
    
inputs = tokenizer(list(test_df['Utterance']), return_tensors="pt", padding=True, truncation=True)
with torch.no_grad():
    outputs = model(**inputs)
    last_hidden_states = outputs.last_hidden_state
    
# Pool embeddings (mean) and convert to DataFrame
mean_embeddings = torch.mean(last_hidden_states, dim=1)
embedding_df = pd.DataFrame(mean_embeddings.numpy())
embedding_df.to_csv("output_embeddings_test.csv", index=False)
    
text_emb_df = pd.read_csv("output_embeddings_test.csv")
X = text_emb_df.values


y_pred1 = model1.predict(X)



In [10]:
y_pred1

array(['neutral', 'neutral', 'neutral', 'joy', 'joy', 'neutral',
       'neutral', 'neutral', 'surprise', 'neutral', 'neutral', 'surprise',
       'neutral', 'surprise', 'neutral', 'neutral', 'neutral', 'neutral',
       'joy', 'neutral', 'joy', 'neutral', 'neutral', 'neutral',
       'neutral', 'neutral', 'neutral', 'joy', 'neutral', 'neutral',
       'anger', 'neutral', 'neutral', 'surprise', 'neutral', 'neutral',
       'neutral', 'neutral', 'surprise', 'neutral', 'surprise',
       'surprise', 'anger', 'joy', 'neutral', 'joy', 'neutral', 'anger',
       'joy', 'neutral', 'neutral', 'neutral', 'neutral', 'joy',
       'neutral', 'joy', 'anger', 'surprise', 'neutral', 'neutral', 'joy',
       'joy', 'joy', 'surprise', 'neutral', 'joy', 'surprise', 'anger',
       'surprise', 'neutral', 'neutral', 'neutral', 'neutral', 'joy',
       'neutral', 'neutral', 'joy', 'surprise', 'neutral', 'neutral',
       'neutral', 'neutral', 'surprise', 'neutral', 'joy', 'anger', 'joy',
       'neutral'

# for audio:


In [11]:
feat_list = []

def reduce_noise(y, sr):
    return nr.reduce_noise(y=y, sr=sr)

def normalize_audio(y):
    return librosa.util.normalize(y)

def trim_silence(y):
    return librosa.effects.trim(y)[0]  # Trimming the silence

def bandpass_filter(y, sr, lowcut=300.0, highcut=3400.0):
    nyquist = 0.5 * sr
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = signal.butter(1, [low, high], btype='band')
    return signal.filtfilt(b, a, y)
    
def extract_features(file_name):
    try:
        y, sr = librosa.load(file_name, sr=None)

        y = reduce_noise(y, sr)  # Reduce noise
        y = normalize_audio(y)  # Normalize volume
        y = trim_silence(y)  # Trim silence
        y = bandpass_filter(y, sr)  # Apply bandpass filter
        features = np.array([])
        mfccs = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13).T, axis=0)
        features = np.hstack((features, mfccs))
        return mfccs
    except Exception as e:
        print(f"Error extracting features from {file_name}: {str(e)}")
        return None
    
    
for root, dirs, files in os.walk(video_dir):
    for file in files:
        if file.endswith(".mp4"):
            file_path = os.path.join(root, file)
            video = VideoFileClip(file_path)
            if os.path.exists(file_path):
                try:
                    #print(f"Processing file: {file_path}")  
                    feature_row = extract_features(file_path)
                    feat_list.append(feature_row)  # Append extracted features
                except Exception as e:
                    print(f"Error processing file {file_path}: {e}")
if feat_list:  
    df = pd.DataFrame(feat_list)
    df.to_csv('extracted_features.csv', index=False)
    print("Features saved to extracted_features.csv")
else:
    print("No features extracted. CSV not created.")




Features saved to extracted_features.csv


In [12]:
mfcc_df = pd.read_csv('/kaggle/working/extracted_features.csv')
mfcc_df

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,-288.281016,173.053031,-81.439179,21.642336,-9.416497,-20.319524,2.987701,-19.221882,-13.737460,-22.959758,-9.639888,-7.741298,-26.569621
1,-434.083649,123.540579,-12.452960,-21.523614,-8.762212,-22.925595,-20.571649,-10.076071,-15.267128,-22.514167,-16.076967,-7.590071,-6.427438
2,-441.886749,79.824012,-27.648004,-15.813201,-8.972926,-12.410840,-8.344969,-14.256541,-12.594261,-10.196259,-11.660397,-11.150313,-9.044853
3,-411.069169,116.349415,-35.914367,-2.458203,20.271546,-14.621329,-19.533682,-5.034328,-8.377897,-18.561195,-14.263538,-10.582928,-8.405179
4,-450.088235,113.448981,-15.337715,-15.342417,2.048335,-8.386977,-11.468304,-7.250701,-5.321934,-8.771061,-7.262328,-4.452344,-7.452931
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,-381.286721,127.367958,-28.907042,-3.852930,6.189430,4.321953,2.969028,-14.245539,-14.565800,-11.952904,-14.710505,-16.945110,-11.142712
996,-496.820749,31.424496,-25.256315,5.053921,4.127785,-11.231469,-7.922420,-6.187952,-0.594216,-1.729757,-5.547633,-2.994170,-2.026336
997,-477.570162,90.444126,-8.272073,-6.295080,0.099327,-11.986740,-8.768549,-8.289945,-14.398389,-13.327586,-10.242979,-12.183804,-9.547850
998,-456.808614,155.201517,-17.221488,-30.436051,-0.679405,-10.592259,-22.477110,-19.083408,-13.278046,-15.205106,-11.948591,-3.605731,-6.319647


In [13]:
mfcc_new = mfcc_df.join(emotions)
mfcc_new

,0,1,2,3,4,5,6,7,8,9,10,11,12,Emotion
0,-288.281016,173.053031,-81.439179,21.642336,-9.416497,-20.319524,2.987701,-19.221882,-13.737460,-22.959758,-9.639888,-7.741298,-26.569621,neutral
1,-434.083649,123.540579,-12.452960,-21.523614,-8.762212,-22.925595,-20.571649,-10.076071,-15.267128,-22.514167,-16.076967,-7.590071,-6.427438,neutral
2,-441.886749,79.824012,-27.648004,-15.813201,-8.972926,-12.410840,-8.344969,-14.256541,-12.594261,-10.196259,-11.660397,-11.150313,-9.044853,neutral
3,-411.069169,116.349415,-35.914367,-2.458203,20.271546,-14.621329,-19.533682,-5.034328,-8.377897,-18.561195,-14.263538,-10.582928,-8.405179,neutral
4,-450.088235,113.448981,-15.337715,-15.342417,2.048335,-8.386977,-11.468304,-7.250701,-5.321934,-8.771061,-7.262328,-4.452344,-7.452931,neutral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,-381.286721,127.367958,-28.907042,-3.852930,6.189430,4.321953,2.969028,-14.245539,-14.565800,-11.952904,-14.710505,-16.945110,-11.142712,neutral
996,-496.820749,31.424496,-25.256315,5.053921,4.127785,-11.231469,-7.922420,-6.187952,-0.594216,-1.729757,-5.547633,-2.994170,-2.026336,joy
997,-477.570162,90.444126,-8.272073,-6.295080,0.099327,-11.986740,-8.768549,-8.289945,-14.398389,-13.327586,-10.242979,-12.183804,-9.547850,joy
998,-456.808614,155.201517,-17.221488,-30.436051,-0.679405,-10.592259,-22.477110,-19.083408,-13.278046,-15.205106,-11.948591,-3.605731,-6.319647,sadness


In [14]:
x1 = mfcc_new.iloc[:, :-2]
y1 = mfcc_new.iloc[:, -1]
X1_train, X1_test, y1_train, y1_test = train_test_split(x1, y1, test_size=0.2, random_state=42)

model2 = MLPClassifier(hidden_layer_sizes=(100, 50),  
                      activation='logistic', 
                      solver='lbfgs', 
                      max_iter=30,  
                      random_state=42)


model2.fit(X1_train, y1_train)

y1_pred = model2.predict(X1_test)

accuracy = accuracy_score(y1_test, y1_pred)
print("Accuracy:", accuracy) 



Accuracy: 0.53


In [15]:
y1_pred

array(['neutral', 'neutral', 'surprise', 'neutral', 'neutral', 'neutral',
       'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral',
       'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral',
       'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral',
       'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral',
       'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral',
       'surprise', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral',
       'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral',
       'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral',
       'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral',
       'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral',
       'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral',
       'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral',
       'neutral', 'neutral', 'neutral', 'neutral'

In [16]:
feat_list = []

    
for root, dirs, files in os.walk(test_video_dir):
    for file in files:
        if file.endswith(".mp4"):
            file_path = os.path.join(root, file)
            video = VideoFileClip(file_path)
            if os.path.exists(file_path):
                try:
                    #print(f"Processing file: {file_path}")  
                    feature_row = extract_features(file_path)
                    feat_list.append(feature_row)  # Append extracted features
                except Exception as e:
                    print(f"Error processing file {file_path}: {e}")
if feat_list:  
    df = pd.DataFrame(feat_list)
    df.to_csv('extracted_features_test.csv', index=False)
    print("Features saved to extracted_features_test.csv")
else:
    print("No features extracted. CSV not created.")


mfcc_df = pd.read_csv('/kaggle/working/extracted_features_test.csv')
x1 = mfcc_df.iloc[:, :-1]


y1_pred1 = model2.predict(x1)

Features saved to extracted_features_test.csv


In [17]:
y1_pred1

array(['neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral',
       'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral',
       'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral',
       'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral',
       'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral',
       'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral',
       'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral',
       'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral',
       'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral',
       'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral',
       'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral',
       'neutral', 'surprise', 'neutral', 'neutral', 'neutral', 'neutral',
       'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral',
       'neutral', 'neutral', 'neutral', 'neutral',

# Combined model - early fusion

In [18]:
combined_features = np.concatenate((X_train, X1_train), axis=1) # Assuming your data is ready
X_test_combined = np.concatenate((X_test, X1_test), axis=1) 

# Train a Combined Model
combined_model = MLPClassifier(hidden_layer_sizes=(100, 50), 
                               activation='tanh', 
                               solver='adam', 
                               max_iter=72, 
                               random_state=42)
combined_model.fit(combined_features, y_train) # Assuming your data is ready

# Make Predictions
y_pred_combined = combined_model.predict(X_test_combined)

# Evaluate Performance
accuracy = accuracy_score(y_test, y_pred_combined)
print("Accuracy of Combined Model:", accuracy)

Accuracy of Combined Model: 0.655


In [19]:
x_test = np.concatenate((X, x1), axis=1) 

y_pred_combined1 = combined_model.predict(x_test)

In [20]:
y_pred_combined1

array(['neutral', 'neutral', 'sadness', 'anger', 'joy', 'neutral',
       'neutral', 'neutral', 'surprise', 'neutral', 'neutral', 'surprise',
       'neutral', 'surprise', 'neutral', 'neutral', 'neutral', 'neutral',
       'joy', 'neutral', 'joy', 'neutral', 'neutral', 'neutral',
       'neutral', 'neutral', 'neutral', 'joy', 'neutral', 'neutral',
       'joy', 'neutral', 'neutral', 'anger', 'neutral', 'joy', 'neutral',
       'joy', 'surprise', 'neutral', 'neutral', 'surprise', 'anger',
       'joy', 'neutral', 'joy', 'neutral', 'neutral', 'joy', 'neutral',
       'neutral', 'neutral', 'neutral', 'joy', 'neutral', 'joy',
       'sadness', 'surprise', 'neutral', 'neutral', 'joy', 'joy', 'joy',
       'surprise', 'joy', 'joy', 'surprise', 'neutral', 'surprise',
       'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral',
       'neutral', 'joy', 'surprise', 'neutral', 'neutral', 'neutral',
       'neutral', 'joy', 'neutral', 'joy', 'anger', 'joy', 'neutral',
       'neutral'

# Combined model - late fusion

In [21]:
label_encoder = LabelEncoder()

# Fit the encoder on the unique class labels and transform predictions
y_pred_encoded = label_encoder.fit_transform(y_pred)
y1_pred_encoded = label_encoder.transform(y1_pred)  # Use the same encoder for consistency

# Stack and transpose the encoded predictions
combined_predictions = np.vstack((y_pred_encoded, y1_pred_encoded)).T

# Calculate the mode along axis=1 for majority voting
final_predictions_encoded, _ = stats.mode(combined_predictions, axis=1)
final_predictions_encoded = final_predictions_encoded.flatten()

# Decode the final predictions back to original class labels
final_predictions = label_encoder.inverse_transform(final_predictions_encoded)

# Calculate accuracy
accuracy = accuracy_score(y_test, final_predictions)
print("Accuracy of Combined Model:", accuracy)

Accuracy of Combined Model: 0.615


# for video:

In [22]:
import os
import cv2
import numpy as np
import pandas as pd
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input


# Initialize ResNet model
model3 = ResNet50(weights='imagenet', include_top=False, pooling='avg')

# Function to extract a fixed number of frames from each video
def extract_frames(video_path, num_frames=10, frame_size=(112, 112)):
    frames = []
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_interval = max(1, total_frames // num_frames)

    while cap.isOpened() and len(frames) < num_frames:
        ret, frame = cap.read()
        if not ret:
            break
        if int(cap.get(cv2.CAP_PROP_POS_FRAMES)) % frame_interval == 0:
            frame = cv2.resize(frame, frame_size)
            frames.append(frame)
    cap.release()
    return np.array(frames)

# Function to extract features in batch
def extract_video_features_batch(video_path, num_frames=10):
    frames = extract_frames(video_path, num_frames)
    frames = preprocess_input(frames)  # Batch preprocess
    features = model3.predict(frames, batch_size=num_frames)
    return np.mean(features, axis=0)  # Average pooling for the frames

# Extract and store features and labels
video_features = []
video_labels = []
emotion_map = {'neutral': 0, 'surprise': 1,'sadness': 2, 'joy': 3,'anger': 4}

# Process each video and track progress
total_videos = len(train_df)
for index, row in train_df.iterrows():
    video_path = row['video_clip_path']
    emotion = row['Emotion']
    print(f"Processing video {index + 1}/{total_videos}: {video_path}")

    features = extract_video_features_batch(video_path)
    video_features.append(features)
    video_labels.append(emotion_map[emotion])

# Convert to numpy arrays for training
video_features = np.array(video_features)
video_labels = np.array(video_labels)

print(f"Extracted video features shape: {video_features.shape}")
print(f"Labels shape: {video_labels.shape}")

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Processing video 1/1000: /kaggle/input/ml-hackathon-ec-campus-set-2/set_2_train/train_data/dia0_utt7.mp4
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Processing video 2/1000: /kaggle/input/ml-hackathon-ec-campus-set-2/set_2_train/train_data/dia0_utt11.mp4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 243ms/step
Processing video 3/1000: /kaggle/input/ml-hackathon-ec-campus-set-2/set_2_train/train_data/dia2_utt8.mp4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 241ms/step
Processing video 4/1000: /kaggle/input/ml-hackathon-ec-campus-set-2/set_2_train/train_data/dia3_utt3.mp4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 239ms/step
Processing video 5/1000: /kaggle/input/ml-hackathon-ec-campus-set-2/set_2_train/train_data/dia3_utt5.mp4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 241ms/step
Processing video 6/1000: /kaggle/input/ml-hackathon-ec-campus-set-2/set_2_train/train_data/dia4_utt1.mp4
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Processing video 7/1000: /kaggle/input/ml-hackathon-ec-campus-set-2/set_2_train/train_data

In [23]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv1D, GlobalMaxPooling1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import LambdaCallback


# One-hot encode the labels
num_classes = 5
video_labels_cat = to_categorical(video_labels, num_classes=num_classes)


In [24]:

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(video_features, video_labels_cat, test_size=0.2, random_state=42)

# Build the CNN model
model4 = Sequential([
    Conv1D(64, kernel_size=3, activation='relu', input_shape=(2048, 1)), 
    GlobalMaxPooling1D(),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

# Compile the model
model4.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Reshape features for Conv1D (from (samples, 2048) to (samples, 2048, 1))
X_train = np.expand_dims(X_train, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)

# Callback to print progress after each epoch
progress_callback = LambdaCallback(
    on_epoch_end=lambda epoch, logs: print(f"Epoch {epoch + 1}/{20} - loss: {logs['loss']:.4f} - accuracy: {logs['accuracy']:.4f} - val_loss: {logs['val_loss']:.4f} - val_accuracy: {logs['val_accuracy']:.4f}")
)

# Train the model with progress callback
history = model4.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test), callbacks=[progress_callback])

# Evaluate the model
test_loss, test_accuracy = model4.evaluate(X_test, y_test)
print(f"Test accuracy: {test_accuracy:.2f}")

Epoch 1/20
23/25 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.2034 - loss: 3.0940Epoch 1/20 - loss: 2.5750 - accuracy: 0.2812 - val_loss: 1.3934 - val_accuracy: 0.5400
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step - accuracy: 0.2124 - loss: 3.0353 - val_accuracy: 0.5400 - val_loss: 1.3934
Epoch 2/20
24/25 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.3307 - loss: 2.1173Epoch 2/20 - loss: 1.8465 - accuracy: 0.3462 - val_loss: 1.3414 - val_accuracy: 0.5400
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.3319 - loss: 2.0965 - val_accuracy: 0.5400 - val_loss: 1.3414
Epoch 3/20
23/25 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.3377 - loss: 1.7569Epoch 3/20 - loss: 1.7294 - accuracy: 0.3487 - val_loss: 1.3626 - val_accuracy: 0.5400
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - accuracy: 0.3391 - loss: 1.7534 - val_accuracy: 0.5400 - val_loss: 1.3626
Epoch 4/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.3973 - loss: 1.5656Epoch 4/20 - loss: 1.5805 - accuracy: 0.4137 - val_loss

In [25]:
predictions_prob = model4.predict(X_test)

predictions_classes = np.argmax(predictions_prob, axis=-1)

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step


In [26]:
predictions_classes

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0])

In [27]:
emotion_map_reverse = {v: k for k, v in emotion_map.items()}
predictions_classes = [emotion_map_reverse[label] for label in predictions_classes]

In [28]:
predictions_classes

['neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'ne

In [29]:
def predict_test_videos(test_df, num_frames=10):
    test_video_features = []
    
    # Extract features for each video in the test set
    for index, row in test_df.iterrows():
        video_path = row['video_clip_path']
        print(f"Processing test video {index + 1}/{len(test_df)}: {video_path}")
        
        features = extract_video_features_batch(video_path, num_frames)
        test_video_features.append(features)
    
    # Convert features to numpy array
    test_video_features = np.array(test_video_features)
    
    # Reshape features for Conv1D input (from (samples, 2048) to (samples, 2048, 1))
    test_video_features = np.expand_dims(test_video_features, axis=-1)
    
    # Predict using the trained model
    predictions = model4.predict(test_video_features)
    
    # Convert predictions to class labels (indices of the max value in each prediction)
    predicted_labels = np.argmax(predictions, axis=1)
    
    # Map predicted labels back to emotion names
    emotion_map_reverse = {v: k for k, v in emotion_map.items()}
    predicted_emotions = [emotion_map_reverse[label] for label in predicted_labels]
    
    return predicted_emotions

# Get predictions for the test dataset
predicted_emotions = predict_test_videos(test_df)

Processing test video 1/100: /kaggle/input/ml-hackathon-ec-campus-set-2/set_2_test/test_data/dia3_utt0.mp4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 258ms/step
Processing test video 2/100: /kaggle/input/ml-hackathon-ec-campus-set-2/set_2_test/test_data/dia5_utt7.mp4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 254ms/step
Processing test video 3/100: /kaggle/input/ml-hackathon-ec-campus-set-2/set_2_test/test_data/dia13_utt2.mp4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 260ms/step
Processing test video 4/100: /kaggle/input/ml-hackathon-ec-campus-set-2/set_2_test/test_data/dia13_utt6.mp4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 260ms/step
Processing test video 5/100: /kaggle/input/ml-hackathon-ec-campus-set-2/set_2_test/test_data/dia14_utt1.mp4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 263ms/step
Processing test video 6/100: /kaggle/input/ml-hackathon-ec-campus-set-2/set_2_test/test_data/dia16_utt0.mp4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 261ms/step
Processing test video 7/100: /kaggle/input/ml-hackathon-ec-campus-set-2/set_2_test/test_data/dia17_utt13.mp4
1/1 ━━━━━━━

In [30]:
predicted_emotions

['neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'ne

# Late fusion - 

In [31]:
class_labels = ['joy', 'sadness', 'anger', 'neutral', 'surprise']

# Create a LabelEncoder to encode class labels
encoder = LabelEncoder()
encoder.fit(class_labels)

LabelEncoder()

In [32]:

# One-hot encode both prediction arrays
enc_emotions = encoder.transform(predictions_classes)
enc_combined = encoder.transform(y_pred_combined)


In [33]:
def to_one_hot(encoded_predictions, num_classes):
    return np.eye(num_classes)[encoded_predictions]

num_classes = len(class_labels)

one_hot_emotions = to_one_hot(enc_emotions, num_classes)
one_hot_combined = to_one_hot(enc_combined, num_classes)

# Assign weights
weight_emotions = 0.1
weight_combined = 0.9

weighted_avg = (one_hot_emotions * weight_emotions + one_hot_combined * weight_combined)

final_encoded_predictions = np.argmax(weighted_avg, axis=1)
final_predictions = encoder.inverse_transform(final_encoded_predictions)


In [34]:
final_predictions

array(['anger', 'anger', 'neutral', 'surprise', 'neutral', 'neutral',
       'neutral', 'neutral', 'neutral', 'surprise', 'anger', 'neutral',
       'neutral', 'sadness', 'neutral', 'surprise', 'surprise', 'neutral',
       'neutral', 'anger', 'neutral', 'neutral', 'neutral', 'joy',
       'neutral', 'anger', 'sadness', 'neutral', 'neutral', 'neutral',
       'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral',
       'neutral', 'neutral', 'neutral', 'joy', 'neutral', 'neutral',
       'joy', 'surprise', 'neutral', 'neutral', 'neutral', 'neutral',
       'neutral', 'neutral', 'anger', 'joy', 'joy', 'joy', 'sadness',
       'neutral', 'joy', 'anger', 'joy', 'surprise', 'neutral', 'neutral',
       'neutral', 'neutral', 'surprise', 'neutral', 'neutral', 'neutral',
       'neutral', 'neutral', 'neutral', 'neutral', 'surprise', 'neutral',
       'neutral', 'sadness', 'joy', 'neutral', 'joy', 'anger', 'joy',
       'joy', 'surprise', 'neutral', 'surprise', 'neutral', 'neutral',

In [35]:

# One-hot encode both prediction arrays
encoded_emotions = encoder.transform(predicted_emotions)
encoded_combined = encoder.transform(y_pred_combined1)

# Convert predictions to one-hot encoding
one_hot_emotions1 = to_one_hot(encoded_emotions, num_classes)
one_hot_combined1 = to_one_hot(encoded_combined, num_classes)

# Calculate the weighted average of the one-hot encoded predictions
weighted_avg1 = (one_hot_emotions1 * weight_emotions + one_hot_combined1 * weight_combined)

# Select the class with the highest weighted average for each sample
final_encoded_predictions1 = np.argmax(weighted_avg1, axis=1)

# Convert the final predictions back to class labels
final_predictions1 = encoder.inverse_transform(final_encoded_predictions1)

print(final_predictions1)

['neutral' 'neutral' 'sadness' 'anger' 'joy' 'neutral' 'neutral' 'neutral'
 'surprise' 'neutral' 'neutral' 'surprise' 'neutral' 'surprise' 'neutral'
 'neutral' 'neutral' 'neutral' 'joy' 'neutral' 'joy' 'neutral' 'neutral'
 'neutral' 'neutral' 'neutral' 'neutral' 'joy' 'neutral' 'neutral' 'joy'
 'neutral' 'neutral' 'anger' 'neutral' 'joy' 'neutral' 'joy' 'surprise'
 'neutral' 'neutral' 'surprise' 'anger' 'joy' 'neutral' 'joy' 'neutral'
 'neutral' 'joy' 'neutral' 'neutral' 'neutral' 'neutral' 'joy' 'neutral'
 'joy' 'sadness' 'surprise' 'neutral' 'neutral' 'joy' 'joy' 'joy'
 'surprise' 'joy' 'joy' 'surprise' 'neutral' 'surprise' 'neutral'
 'neutral' 'neutral' 'neutral' 'neutral' 'neutral' 'neutral' 'joy'
 'surprise' 'neutral' 'neutral' 'neutral' 'neutral' 'joy' 'neutral' 'joy'
 'anger' 'joy' 'neutral' 'neutral' 'neutral' 'neutral' 'neutral' 'neutral'
 'joy' 'neutral' 'neutral' 'neutral' 'neutral' 'neutral' 'joy']


# For submission:

In [36]:
all_preds = final_predictions1
all_ids = test_df["Sr No."]
submission_df = pd.DataFrame({
        'Sr No.': all_ids,
        'Emotion': all_preds
    })
    
# Save the DataFrame to CSV
submission_df.to_csv("submission.csv", index=False)